# HIPÓTESIS: En qué pais es menos arriesgado salir de pesca

##### Importo bibliotecas y el archivo csv

In [14]:
import pandas as pd
import numpy as np
import statistics as st
import matplotlib.pyplot as plt
import re
df = pd.read_csv("input/attacks.csv",encoding='ISO-8859-1')
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


##### Limpio el DF corrigiendo errores para facilitar el análisis de los datos

In [15]:
print(df.shape)
df.columns

(25723, 24)


Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

##### Corrijo los nombres de las columnas

In [16]:
df_col = df.rename(columns={'Sex ':'Sex','Species ':'Species','Fatal (Y/N)':'Fatal'})

######  Analizo los elementos nulos de cada columna

In [17]:
null_cols = df_col.isnull().sum()
null_cols

Case Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex                       19986
Age                       22252
Injury                    19449
Fatal                     19960
Time                      22775
Species                   22259
Investigator or Source    19438
pdf                       19421
href formula              19422
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Unnamed: 22               25722
Unnamed: 23               25721
dtype: int64

##### Elimino las columnas que no necesito
- Elimino las columnas 'Unnamed:22' y 'Unnamed:23' porque sus valores son 'Nan'.
- Elimino 'href formula', 'href', 'pdf', 'Investigator or source' y 'original order' porque no considero que su contenido aporte información para mi análisis.
- Elimino la columna 'pdf' ya que sus valores son los nombres de los archivos pero no puedo acceder a ellos.
- Elimino 'Case Number', 'Case Numer.1' y 'Case Number.2' porque contienen la misma info que podemos encontrar en 'Date'

In [18]:
clean1 = df_col.drop(columns=['Case Number','Unnamed: 22', 'Unnamed: 23', 'Investigator or Source','pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order'])
display(clean1.head())
clean1.shape

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species
0,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark
1,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN
2,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN
3,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark
4,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m"


(25723, 14)

In [30]:
clean1['Fatal'] = clean1['Fatal'].str.replace(' N', 'N')
clean1 = clean1[clean1.Year != 0]
clean2 = clean1
display(clean2.head())

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species
0,Jun,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark
1,Jun,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN
2,Jun,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN
3,Jun,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark
4,Jun,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m"


##### Limpio la columna 'Date' para sacar los meses y converirla en 'Month'

In [25]:
clean2['Date'] = clean1['Date'].str.replace('-', ' ')
clean2['Date'] = clean1['Date'].str.replace('\d+', '')
clean2['Date'] = clean1['Date'].str.replace('Reported', '')
clean2['Date'] = clean1['Date'].str.replace('Early', '')
clean2['Date'] = clean1['Date'].str.replace('Late', '')
clean2['Date'] = clean1['Date'].str.replace(' ', '')
clean2['Date'] = clean1['Date'].str.replace('Summer, summer, summerof ', '')

In [29]:
clean3 = clean2[clean2.Date != ""]
clean3 = clean3[clean3["Date"].isin(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug', 'Sep','Oct','Nov','Dec'])]
clean3 = clean3.rename(columns={'Date':'Month'})
display(clean3.head())

,Month,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species
0,Jun,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark
1,Jun,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN
2,Jun,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN
3,Jun,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark
4,Jun,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m"


##### Corrijo los valores erroneos de la columna 'Fatal'

In [32]:
clean3['Fatal'].value_counts()

N          3994
Y          1173
UNKNOWN      44
2017          1
M             1
Name: Fatal, dtype: int64

In [33]:
clean4 = clean3[clean3.Fatal != "UNKNOWN"]
clean4 = clean4[clean4.Fatal != "2017"]
clean4 = clean4[clean4.Fatal != "M"]
display(clean4.head())

,Month,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal,Time,Species
0,Jun,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark
1,Jun,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN
2,Jun,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN
3,Jun,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark
4,Jun,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m"


##### En qué zonas hay más casos?

In [35]:
clean4['Country'].value_counts()

USA                       2103
AUSTRALIA                 1245
SOUTH AFRICA               543
NEW ZEALAND                115
BRAZIL                     109
                          ... 
Seychelles                   1
SUDAN?                       1
SAN DOMINGO                  1
BRITISH VIRGIN ISLANDS       1
PERU                         1
Name: Country, Length: 184, dtype: int64

##### Cuáles son las actividades más afectadas?

In [37]:
clean4['Activity'].value_counts()

Surfing                                              946
Swimming                                             805
Fishing                                              392
Spearfishing                                         298
Bathing                                              144
                                                    ... 
Diving, feeding sharks                                 1
Fishing schooner Jose Luis foundered                   1
Swimming in waist-deep water                           1
Scuba diving & filming                                 1
Working on the bridge when he fell into the river      1
Name: Activity, Length: 1360, dtype: int64

##### DF limpio
Elimino columnas que no voy a necesitar para el análisis y me quedo con un nuevo DF limpio
Para el análisis se van a comparar los tres paises con mayor número de registros de ataques (USA, AUSTRALIA y SOUTH AFRICA) y se van a eliminar columnas que no aportan información relevante como los datos relacionados con el usuario.

In [38]:
df_clean = clean4.drop(columns=['Name','Age','Time','Type','Sex','Species'])
df_clean = df_clean[(df_clean.Country=='USA') | (df_clean.Country=='AUSTRALIA') | (df_clean.Country=='SOUTH AFRICA')]

In [39]:
df_clean['Area']=df_clean['Area'].replace(np.nan,'')
df_clean['Location']=df_clean['Location'].replace(np.nan,'')
df_clean['Activity']=df_clean['Activity'].replace(np.nan,'')
df_clean['Injury']=df_clean['Injury'].replace(np.nan,'')
df_clean['Fatal']=df_clean['Fatal'].replace(np.nan,'')

In [40]:
df_clean.drop(df_clean[df_clean.Year.isnull()].index, inplace=True)

In [41]:
display(df_clean.head())

,Month,Year,Country,Area,Location,Activity,Injury,Fatal
0,Jun,2018.0,USA,California,"Oceanside, San Diego County",Paddling,"No injury to occupant, outrigger canoe and pad...",N
1,Jun,2018.0,USA,Georgia,"St. Simon Island, Glynn County",Standing,Minor injury to left thigh,N
2,Jun,2018.0,USA,Hawaii,"Habush, Oahu",Surfing,Injury to left lower leg from surfboard skeg,N
3,Jun,2018.0,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,Minor injury to lower leg,N
5,Jun,2018.0,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,"No injury, board bitten",N


##### Análisis estadístico de los años de los ataques registrados

In [42]:
df_clean["Year"].describe()

count    3890.000000
mean     1975.597429
std        40.243651
min      1751.000000
25%      1956.000000
50%      1989.000000
75%      2007.000000
max      2018.000000
Name: Year, dtype: float64

###### Para el análisis nos quedamos con los datos desde 1960, aproximadamente el percentil 25, ya que en los 200 años anteriores no hay casi registros

In [43]:
df_clean_year = df_clean.drop(df_clean[df_clean['Year']<1960].index)

In [44]:
df_clean_year["Year"].describe()

count    2818.000000
mean     1996.183818
std        17.151011
min      1960.000000
25%      1984.000000
50%      2001.000000
75%      2011.000000
max      2018.000000
Name: Year, dtype: float64

In [45]:
bins = list(range(1959,2029,10))
labels = list(range(1960,2020,10))
df_clean_year["Decade"] = pd.cut(df_clean_year.Year,bins=bins,labels=labels)

In [46]:
shark=df_clean_year[['Month','Year','Decade','Country','Area','Location','Activity','Injury','Fatal']]
display(shark.head())

,Month,Year,Decade,Country,Area,Location,Activity,Injury,Fatal
0,Jun,2018.0,2010,USA,California,"Oceanside, San Diego County",Paddling,"No injury to occupant, outrigger canoe and pad...",N
1,Jun,2018.0,2010,USA,Georgia,"St. Simon Island, Glynn County",Standing,Minor injury to left thigh,N
2,Jun,2018.0,2010,USA,Hawaii,"Habush, Oahu",Surfing,Injury to left lower leg from surfboard skeg,N
3,Jun,2018.0,2010,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,Minor injury to lower leg,N
5,Jun,2018.0,2010,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,"No injury, board bitten",N


In [47]:
shark.to_csv("output/sharks.csv")

In [48]:
shark['Activity'].value_counts()

Surfing                            826
Swimming                           328
                                   157
Spearfishing                       141
Fishing                            134
                                  ... 
Playing in knee-deep water           1
Diving for lobsters                  1
Diving in aquarium display tank      1
Motor boat Rebel Belle lost          1
Holding onto an inflatable boat      1
Name: Activity, Length: 542, dtype: int64

##### Importo un archivo con los datos demográficos de Australia (AUS), USA (USA) y South Africa (SAF)

In [49]:
poblacion = pd.read_excel("input/Poblacion.xlsx")
display(poblacion.head())

,Fecha,Densidad AUS,Hombres AUS,Mujeres AUS,Poblacion AUS,Densidad USA,Hombres USA,Mujeres USA,Poblacion USA,Densidad SAF,Hombres SAF,Mujeres SAF,Poblacion SAF
0,2018,3.0,12441473,12541215,25169000,33,161644928,165042573,327352000,48,28488734,29290888,57939000
1,2017,3.0,12252228,12349632,24775000,33,160791853,164193686,325326000,47,28116850,28883601,57098000
2,2016,3.0,12049619,12141288,24390000,33,159761286,163180025,323127513,46,27733742,28469912,56252000
3,2015,3.0,11867302,11948693,23985000,33,158591855,162043308,320896618,45,27337003,28049364,55407000
4,2014,3.0,11705167,11770519,23640000,32,157397705,160903303,318563456,45,26924942,27621049,54574000


In [50]:
crecAUS=round(max(poblacion['Poblacion AUS'])/min(poblacion['Poblacion AUS'])-1,2)
crecUSA=round(max(poblacion['Poblacion USA'])/min(poblacion['Poblacion USA'])-1,2)
crecSAF=round(max(poblacion['Poblacion SAF'])/min(poblacion['Poblacion SAF'])-1,2)

print("Crecimiento población AUS de 1960 a 2018:", crecAUS)
print("Crecimiento población USA de 1960 a 2018:", crecUSA)
print("Crecimiento población SAF de 1960 a 2018:", crecSAF)


Crecimiento población AUS de 1960 a 2018: 1.45
Crecimiento población USA de 1960 a 2018: 0.82
Crecimiento población SAF de 1960 a 2018: 2.39


In [51]:
# Población media
pobAUS=int(st.mean(poblacion['Poblacion AUS']))
pobUSA=int(st.mean(poblacion['Poblacion USA']))
pobSAF=int(st.mean(poblacion['Poblacion SAF']))

In [52]:
# Nº Ataques
ataquesUSA=shark[shark['Country']=='USA'].value_counts().sum()
ataquesAUS=shark[shark['Country']=='AUSTRALIA'].value_counts().sum()
ataquesSAF=shark[shark['Country']=='SOUTH AFRICA'].value_counts().sum()

In [53]:
# Resultados de los ataques
fatalUSA=(shark[(shark['Country']=='USA')&(shark['Fatal']=='N')].value_counts().sum())
fatalAUS=(shark[(shark['Country']=='AUSTRALIA')&(shark['Fatal']=='N')].value_counts().sum())
fatalSAF=(shark[(shark['Country']=='SOUTH AFRICA')&(shark['Fatal']=='N')].value_counts().sum())

nofatalUSA=(shark[(shark['Country']=='USA')&(shark['Fatal']=='Y')].value_counts().sum())
nofatalAUS=(shark[(shark['Country']=='AUSTRALIA')&(shark['Fatal']=='Y')].value_counts().sum())
nofatalSAF=(shark[(shark['Country']=='SOUTH AFRICA')&(shark['Fatal']=='Y')].value_counts().sum())

unkUSA=(shark[(shark['Country']=='USA')&(shark['Fatal']=='')].value_counts().sum())
unkAUS=(shark[(shark['Country']=='AUSTRALIA')&(shark['Fatal']=='')].value_counts().sum())
unkSAF=(shark[(shark['Country']=='SOUTH AFRICA')&(shark['Fatal']=='')].value_counts().sum())

In [54]:
# Ataques Fishing and Spearfishing
atUSA_f=shark[(shark['Country']=='USA')&((shark['Activity']=='Fishing')|(shark['Activity']=='Spearfishing'))].value_counts().sum()
atAUS_f=shark[(shark['Country']=='AUSTRALIA')&((shark['Activity']=='Fishing')|(shark['Activity']=='Spearfishing'))].value_counts().sum()
atSAF_f=shark[(shark['Country']=='SOUTH AFRICA')&((shark['Activity']=='Fishing')|(shark['Activity']=='Spearfishing'))].value_counts().sum()

In [55]:
'''
Volumen de pesca:
    - EEUU: 6,5 millones de toneladas - 23Kg per cápita al año
    - Australia: 769 mil toneladas - 20Kg per cápita al año
    - Sudáfrica: 2,3 millones de toneladas / 33 mil toneladas - 19Kg per cápita al año
    (fuente: http://www.fao.org/3/w3265s/w3265s04.htm)
'''
pescaUSA=6500000
pescaAUS=769000
pescaSAF=2300000

In [56]:
analisis={'USA':
          {'Poblacion':pobUSA,
           'Ataques':ataquesUSA,
           '%Ataques*1000':round(ataquesUSA/pobUSA*100*1000,2),
           '%Fatal':round(fatalUSA/ataquesUSA*100,2),
           '%NO Fatal':round(nofatalUSA/ataquesUSA*100,2),
           '%Unknown':round(unkUSA/ataquesUSA*100,2),
           'Fishing and Spearfishing':atUSA_f,
           '%Fishing and Spearf.':round(atUSA_f/ataquesUSA*100,2),
           'Vol Pesca (Tn)':pescaUSA
          },
         'AUSTRALIA':
         {'Poblacion':pobAUS,
           'Ataques':ataquesAUS,
          '%Ataques*1000':round(ataquesAUS/pobAUS*100*1000,2),
           '%Fatal':round(fatalAUS/ataquesAUS*100,2),
           '%NO Fatal':round(nofatalAUS/ataquesAUS*100,2),
           '%Unknown':round(unkAUS/ataquesAUS*100,2),
            'Fishing and Spearfishing':atAUS_f,
          '%Fishing and Spearf.':round(atAUS_f/ataquesAUS*100,2),
          'Vol Pesca (Tn)':pescaAUS,
         },
          'SOUTH AFRICA':
          {'Poblacion':pobSAF,
           'Ataques':ataquesSAF,
           '%Ataques*1000':round(ataquesSAF/pobSAF*100*1000,2),
           '%Fatal':round(fatalSAF/ataquesSAF*100,2),
           '%NO Fatal':round(nofatalSAF/ataquesSAF*100,2),
           '%Unknown':round(unkSAF/ataquesSAF*100,2),
            'Fishing and Spearfishing':atSAF_f,
           '%Fishing and Spearf.':round(atSAF_f/ataquesSAF*100,2),
           'Vol Pesca (Tn)':pescaSAF
          }
         }
print(analisis)

{'USA': {'Poblacion': 252276564, 'Ataques': 1714, '%Ataques*1000': 0.68, '%Fatal': 88.8, '%NO Fatal': 3.5, '%Unknown': 7.7, 'Fishing and Spearfishing': 89, '%Fishing and Spearf.': 5.19, 'Vol Pesca (Tn)': 6500000}, 'AUSTRALIA': {'Poblacion': 17021652, 'Ataques': 680, '%Ataques*1000': 3.99, '%Fatal': 80.74, '%NO Fatal': 11.18, '%Unknown': 8.09, 'Fishing and Spearfishing': 111, '%Fishing and Spearf.': 16.32, 'Vol Pesca (Tn)': 769000}, 'SOUTH AFRICA': {'Poblacion': 36311664, 'Ataques': 424, '%Ataques*1000': 1.17, '%Fatal': 78.3, '%NO Fatal': 11.08, '%Unknown': 10.61, 'Fishing and Spearfishing': 75, '%Fishing and Spearf.': 17.69, 'Vol Pesca (Tn)': 2300000}}


In [57]:
df_anal = pd.DataFrame([key for key in analisis.keys()],columns=['Country'])
df_anal['Poblacion'] = [value['Poblacion'] for value in analisis.values()]
df_anal['Ataques'] = [value['Ataques'] for value in analisis.values()]
df_anal['%Ataques*1000'] = [value['%Ataques*1000'] for value in analisis.values()]
df_anal['%Fatal'] = [value['%Fatal'] for value in analisis.values()]
df_anal['%NO Fatal'] = [value['%NO Fatal'] for value in analisis.values()]
df_anal['%Unknown'] = [value['%Unknown'] for value in analisis.values()]
#df_anal['Fishing and Spearfishing'] = [value['Fishing and Spearfishing'] for value in analisis.values()]
df_anal['%Fishing and Spearf.'] = [value['%Fishing and Spearf.'] for value in analisis.values()]
df_anal['Vol Pesca (Tn)'] = [value['Vol Pesca (Tn)'] for value in analisis.values()]
df_anal

,Country,Poblacion,Ataques,%Ataques*1000,%Fatal,%NO Fatal,%Unknown,%Fishing and Spearf.,Vol Pesca (Tn)
0,USA,252276564,1714,0.68,88.80,3.50,7.70,5.19,6500000
1,AUSTRALIA,17021652,680,3.99,80.74,11.18,8.09,16.32,769000
2,SOUTH AFRICA,36311664,424,1.17,78.30,11.08,10.61,17.69,2300000


##### Conclusiones:
En primer lugar, comentar que se ha reducido el análisis a Estados Unidos, Australia y Sudáfrica, ya que son las zonas en las que hay mayor número de ataques de tiburones registrados.
Estos registros se remontaban al año 1751, pero durante los primeros 200 años había muy pocos casos, por lo que el análisis se ha hecho teniendo en cuenta los casos desde 1960 (hasta 2018).

- Datos de la tabla:
    - Country: Diferencia las zonas con mayor número de ataques (EEUU, Australia y Sudáfrica)
    - Población: Se ha cogido como referencia la media de la población entre 1960 y 2018
    - Ataques: Número de ataques registrados
    - % Ataques*1000: Porcentaje de ataques sobre la población media (por mil)
    - % Fatal: Porcentaje de muertes por ataques
    - % NO Fatal: Porcentaje supervivientes despues de un ataque
    - % Unknown: Porcentaje unknown despues de un ataque
    - % Fishing and Spearfishing: Porcentaje de ataques relacionados con la pesca sobre el total
        
- Conclusiones:
    - La población es mayor en EEUU, luego Sudáfrica y luego Australia, sin embargo la zona donde más atacan los tiburones es en Australia, seguido de Sudáfrica y terminando por EEUU
    - Teniendo en cuenta los ataques de cada zona, aún habiendo más casos en Australia, la zona donde más atacan a pescadores es en Sudáfrica
    - Volumen de pesca:
        - EEUU: 6,5 millones de toneladas - 23Kg per cápita al año
        - Australia: 769 mil toneladas - 20Kg per cápita al año
        - Sudáfrica: 2,3 millones de toneladas / 33 mil toneladas - 19Kg per cápita al año
    - Donde más volumen de pesca existe es en EEUU, y es donde menos ataques a pescadores existen, por lo que es la zona más segura para pescar, dentro de las zonas donde atacan los tiburones. 
       Sudáfrica es la zona donde más ataques a pescadores hay, pero teniendo en cuenta el ratio de ataques con respecto al volumen de pesca, podemos comprobar que la zona más peligrosa para realizar esta actividad es Australia.
    
        